In [1]:
import csv
import rasterio
import os
import numpy as np
from pathlib import Path, PurePath

In [2]:
import os
import inspect
import dl_toolbox
inspect.getfile(dl_toolbox)

'/d/pfournie/dl_toolbox/dl_toolbox/__init__.py'

In [3]:
if os.uname().nodename == 'WDTIS890Z': 
    data_root = Path('/mnt/d/pfournie/Documents/data')
    home = Path('/home/pfournie')
    save_root = data_root / 'outputs'
elif os.uname().nodename == 'qdtis056z': 
    data_root = Path('/data')
    home = Path('/d/pfournie')
    save_root = data_root / 'outputs'
else:
    data_root = Path('/work/OT/ai4geo/DATA/DATASETS')
    #data_root = Path(os.environ['TMPDIR'])
    home = Path('/home/eh/fournip')
    save_root = Path('/work/OT/ai4usr/fournip') / 'outputs'

In [8]:
 with rasterio.open(data_root/'miniworld_tif/christchurch/train/0_x.tif', 'r') as file:
    image = file.read(out_dtype=np.float32, indexes=[1,2,3])
np.max(image)

255.0

In [13]:
# split airs
data_path = data_root / 'miniworld_tif'
with open(Path.home() / f'dl_toolbox/dl_toolbox/datamodules/splits/split_christchurch_filtered.csv', 'r') as src:
    
    reader = csv.reader(src)
    next(reader)
    
    with open(Path.home() / f'dl_toolbox/dl_toolbox/datamodules/airs_50cm.csv', 'w+', newline='') as dst:
        
        writer = csv.writer(dst)
        writer.writerow(
            [
                'city',
                'tile_id',
                'img_path',
                'label_path',
                'x0',
                'y0',
                'patch_width',
                'patch_height',
                'fold_id',
                'min_vals',
                'max_vals'
            ]
        )
        
        i = 0
        for row in reader:
            
            _, _, image_path, label_path, x0, y0, w, h, _ = row[:9]
            p_img = Path('christchurch') / image_path
            p_lbl = Path('christchurch') / label_path
            with rasterio.open(data_path/p_lbl, 'r') as file:
                label = file.read(out_dtype=np.float32)
                #print(p_lbl, np.mean(label))
                if  np.mean(label) > 1:
                    writer.writerow(
                        [
                            'Airs',
                            i,
                            p_img,
                            p_lbl,
                            0,
                            0,
                            w,
                            h,
                            i,
                            str((0., 0., 0.)),
                            str((255., 255., 255.))
                        ]
                    )
                    i+=1

In [ ]:
# Split SEMCITY   
data_path = data_root / 'SemCity-Toulouse-bench'

with open(Path.home() / f'dl_toolbox/dl_toolbox/datamodules/semcity_16tiles.csv', 'w+', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(
        [
            'city',
            'tile_id',
            'img_path',
            'label_path',
            'x0',
            'y0',
            'patch_width',
            'patch_height',
            'fold_id',
            'min_vals',
            'max_vals'
        ]
    )
    for i in range(16): 
        image_path = 'img_multispec_05/TLS_BDSD_M/TLS_BDSD_M_' + f'{i+1:02}' + '.tif'
        label_path = 'semantic_05/TLS_GT/TLS_GT_' + f'{i+1:02}' + '_1.tif'
        with rasterio.open(data_path/image_path) as f:
            height, width = f.shape
            stats = [f.statistics(bidx=i, approx=True, clear_cache=False) for i in range(1, f.count+1)]
            writer.writerow(
                [
                    'Semcity',
                    i,
                    image_path,
                    label_path,
                    0,
                    0,
                    width,
                    height,
                    i,
                    str(tuple([round(s.min,3) for s in stats])),
                    str(tuple([round(s.max,3) for s in stats]))
                ]
            )

In [ ]:
with rasterio.open(digitanie/'Toulouse'/f'Toulouse_EPSG32631_4.tif') as f:
    print('small tile bounds : ', f.bounds)
    x = 373000
    y = 4828000
    print(f'point of coords {(x,y)} at indices : ', f.index(x,y))
    print('point value : ', next(f.sample([(x,y)])))
    
with rasterio.open(digitanie/'Toulouse'/f'Toulouse.tif') as f:
    print(f'point of coords {(x,y)} at indices {f.index(x,y)} in big tile has value', next(f.sample([(x,y)])))


In [ ]:
src = rasterio.open(Path('/work/OT/ai4geo/DATA/DATASETS/DIGITANIE/Toulouse/COS9/Toulouse_4.tif'))
img = src.read(out_dtype=np.uint8)
print(np.unique(img.reshape(img.shape[0], -1), axis=1, return_counts=True))

In [ ]:
# Split DIGITANIE ALL

digitanie = data_root / 'DIGITANIE'

cities = [
    'Arcachon',
    'Brisbane',
    'Can-Tho',
    'Lagos',
    'Maros',
    'Munich',
    'Paris',
    'Rio-Janeiro',
    'Strasbourg',
    'Toulouse',
    'Biarritz',
    'Buenos-Aires',
    'Helsinki',
    'Le-Caire',
    'Montpellier',
    'Nantes',
    'Port-Elisabeth',
    'Shanghai',
    'Tianjin'
]

i = 0
with open(Path.home() / f'dl_toolbox/dl_toolbox/datamodules/digitanie_all.csv', 'w+', newline='') as csvfile:

    writer = csv.writer(csvfile)
    writer.writerow(
        [
            'city',
            'tile_id',
            'img_path',
            'label_path',
            'x0',
            'y0',
            'patch_width',
            'patch_height',
            'fold_id',
            'min_vals',
            'max_vals'
        ]
    )
    
    for city in cities:
        print(city)
        # Opening and computing stats and writing from big city image
        big_tile_path = digitanie/city/f'{city}.tif'        
        with rasterio.open(big_tile_path) as f:
            height, width = f.shape
            big_tile = f.read(window=rasterio.windows.Window(20000, 20000, 10000, 10000))[..., ::5, ::5]
            p2, p98 = np.percentile(big_tile, (2, 98), axis=(1,2))
            print(p2)
            print(p98)
            #stats = [f.statistics(bidx=i, approx=True, clear_cache=False) for i in range(1, f.count+1)]
        writer.writerow(
            [
                f'Digitanie',
                0,
                big_tile_path.relative_to(digitanie),
                'none',
                0,
                0,
                width,
                height,
                i,
                str(tuple(p2)),
                str(tuple(p98))
            ]
        ) 
        i+=1

        # List of small tiles
        img_paths = sorted(
            [path.relative_to(digitanie) for path in Path(digitanie/city).glob('*_[0-9].tif')], 
            key=lambda x: int(x.stem.split('_')[-1])
        )
        msk_paths = sorted(
            [path.relative_to(digitanie) for path in Path(digitanie/city/'COS9').glob('*.tif')],
            key=lambda x: int(x.stem.split('_')[1][0])
        )
        
        # Creating the csv
        for img, msk in zip(img_paths, msk_paths):
            with rasterio.open(digitanie/img) as f:
                height, width = f.shape
                tile = f.read()
                p2, p98 = np.percentile(tile, (2, 98), axis=(1,2))
                print(p2)
                print(p98)
                #stats = [f.statistics(bidx=i, approx=True, clear_cache=False) for i in range(1, f.count+1)]
                writer.writerow(
                    [
                        f'Digitanie',
                        0,
                        img,
                        msk,
                        0,
                        0,
                        width,
                        height,
                        i,
                        str(tuple(p2)),
                        str(tuple(p98))
                    ]
                )
                i+=1


In [ ]:
if os.uname().nodename == 'WDTIS890Z':
    data_path = Path('/mnt/d/pfournie/Documents/data')
elif os.uname().nodename == 'qdtis056z':
    data_path = Path('/data')
else:
    data_path = Path('/work/OT/ai4geo/DATA/DATASETS')
    
digitanie = data_path / 'DIGITANIE'
cities = ['Toulouse']
with open(Path.home() / f'dl_toolbox/dl_toolbox/datamodules/digitanie_toulouse_big_2.csv', 'w+', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(
        [
            'city',
            'tile_id',
            'img_path',
            'label_path',
            'x0',
            'y0',
            'patch_width',
            'patch_height',
            'fold_id',
            'min_vals',
            'max_vals'
        ]
    )
    for i, city in enumerate(cities):
        big_tile_path = digitanie/city/f'{city}.tif'        
        with rasterio.open(big_tile_path) as f:
            height, width = f.shape
            stats = [f.statistics(bidx=i, approx=True, clear_cache=False) for i in range(1, f.count+1)]
            writer.writerow(
                [
                    f'Digitanie',
                    i,
                    big_tile_path.relative_to(digitanie),
                    'none',
                    0,
                    0,
                    width,
                    height,
                    i,
                    str(tuple([round(s.min,3) for s in stats])),
                    str(tuple([round(s.max,3) for s in stats]))
                ]
            )